# Document-at-a-time Query Processing

Implement document-at-a-time query processing using a simple scoring function.

In [1]:
import ipytest
import pytest

from typing import Dict, List, Tuple
from collections import Counter

ipytest.autoconfig()

### Inverted index

For simplicity, the inverted index for the document collection is given as a dictionary, with a terms as keys and posting lists as values. Each posting is a (document ID, term frequency) tuple.

In [2]:
index = {
    "beijing": [        (1, 1),                 (4, 1)],
    "dish":    [        (1, 1),                 (4, 1)],
    "duck":    [(0, 3), (1, 2), (2, 2),         (4, 1)],
    "rabbit":  [                (2, 1), (3, 1)        ],
    "recipe":  [                (2, 1), (3, 1), (4, 1)]
}

### Document lengths

The length of each document is provided in a list (Normally, this information would be present in a document index).



In [3]:
doc_len = [3, 4, 4, 2, 4]

### Document-at-a-time scoring

The retrieval function we use is the following:

$$score(q,d) = \sum_{t \in q} w_{t,d} \times w_{t,q}$$

where $w_{t,d}$ and $w_{t,q}$ are length-normalized term frequencies, i.e., $w_{t,d} = \frac{c_{t,d}}{|d|}$, where $c_{t,d}$ is the number of occurrences of term $t$ in document $d$ and $|d|$ is the document length, i.e., the total number of terms. Similarly for the query.

We utilize the fact that the posting lists are ordered by document ID. Then, it's enough to iterate in parallel through term query term's posting list, and score the minimum docid at each iteration. We keep a pointer for each query term and we move it forward every time a docid is scored.


We use an helper function that, given a dict of posting list pointers `pos`, returns the minimum docid. If all pointers go beyond the end of the corresponding posting list, we return `num_docs`, a special docid that it is guaranteed to not appear in any posting list.

In [34]:
def min_docid(index: Dict[str, List[Tuple[int, int]]], 
              pos: Dict[str, int],
              num_docs: int) -> int:
    """Returns the minimum docid across posting lists.
    
    Args:
        index: Dict holding the inverted index.
        pos: the current positions in the query posting lists.
        num_docs: the number of indexed documents.
    
    Returns:
        the minimum docid across the posting lists, or num_docs if all 
        postings in all posting lists have been processed.
    """
    min_docid = num_docs

    # For each query term posting list.
    for term, pointer in pos.items():
         # TODO: check if the posting list contains a valid docid; if so, update min_docid
        docid = index[term][pointer][0]
        if min_docid > docid:
            min_docid = docid

    for term, pointer in pos.items():
        print(term)
        print(pointer)
        if index[term][pointer][0] == min_docid and pointer < len(index[term]):
            pos[term] = pointer + 1
            

    # Return the min docid computed.
    return min_docid

In [32]:
def score_collection(index: Dict[str, List[Tuple[int, int]]], 
                      doc_len: List[int], 
                      query: str) -> List[Tuple[int, float]]:
    """Scores all documents in the collection.
    
    Args:
        index: Dict holding the inverted index.
        doc_len: List with document lengths.
        query: Search query.
    
    Returns:
        List with (document_id, score) tuples, ordered by score desc.
    """
    
    # Turns the query string into a "term: freq" dictionary.
    query_freqs = dict(Counter(query.split()))

    # Computes query length (i.e., sum of all query term frequencies).
    query_len = sum(query_freqs.values())

    doc_scores = {}  # Holds the final document scores (this should be a priority list, but for simplicity we use a dictionary here).
    
    pos = {term: 0 for term in query_freqs}  # Holds a pointer for each query term's posting list.
    
    # The starting docid.
    docid = min_docid(index, pos, len(doc_len))
    # While the posting lists have not been completely traversed.
    while docid != len(doc_len):
        score = 0.0
        # TODO: DAAT scoring algorithm
        for term, pointer in pos.items():
            back_pointer = pointer - 1
            if pointer == 0 or index[term][back_pointer][0] != docid:
                continue
            print("processo")
            weight = index[term][back_pointer][1] / doc_len[docid]
            partial_score = weight * (query_freqs[term] / query_len)
            score += partial_score
        doc_scores[docid] = score
        print(doc_scores)
        # Update the docid.
        break
        docid = min_docid(index, pos, len(doc_len))
    # TODO: return doc_scores sorted
    return sorted(doc_scores.items(), key=lambda item: item[1],reverse=True)


#### Tests

In [35]:
%%ipytest

def test_scoring():
    scores = score_collection(index, doc_len, "beijing duck recipe")    
    assert scores[0][0] == 0
    assert scores[0][1] == pytest.approx(1/3, rel=1e-2)
    assert scores[2][0] == 2
    assert scores[2][1] == pytest.approx(1/4, rel=1e-2)
    assert scores[4][0] == 3
    assert scores[4][1] == pytest.approx(1/6, rel=1e-2)


F                                                                                            [100%]
============================================= FAILURES =============================================
___________________________________________ test_scoring ___________________________________________

    def test_scoring():
        scores = score_collection(index, doc_len, "beijing duck recipe")
        assert scores[0][0] == 0
        assert scores[0][1] == pytest.approx(1/3, rel=1e-2)
>       assert scores[2][0] == 2
E       IndexError: list index out of range

/var/folders/jq/sxfhvkbn11s0jzgvkdhwp6v00000gn/T/ipykernel_3440/224187975.py:5: IndexError
--------------------------------------- Captured stdout call ---------------------------------------
beijing
0
duck
0
recipe
0
processo
{0: 0.3333333333333333}
===================================== short test summary info ======================================
FAILED tmppguo9ixa.py::test_scoring - IndexError: list index out of range
1 f